In [1]:
from pathlib import Path
from validphys.loader import _get_nnpdf_profile
from validphys.api import API
import numpy as np
import pandas as pd
from validphys.convolution import central_predictions

profile = _get_nnpdf_profile()
#yaml_db = Path(profile["data_path"]) / "yamldb"

The `yaml_db` folder is a temporary thing as it contains files that look like:

```yaml
conversion_factor: 1.0
operands:
- - NMC_NC_EM_D_F2
- - NMC_NC_EM_P_F2
operation: RATIO
target_dataset: NMCPD
```

This information will eventually be part of the new commondata format of course.

The `operation` is applied to the first level of the list while the second level is just concatenated. This is necessary since `pineappl` fktables might contain one layer of concatenation which is already done for the "classic" fktables.

The `pineappl` fktables will live inside the appropiate `theory_xxx` folder `/pineappls`.

In [7]:
# Test them all
if True:
    from yaml import safe_load
    pdf = API.pdf(pdf="NNPDF40_nnlo_as_01180")
    all_res = []
    # Reference here a NNPDF40 runcard to read up all datasets
    #nnpdf40_runcard = safe_load(Path("/home/juacrumar/NNPDF-testing/nnpdf/n3fit/NNPDF40_with_pineappl.yml").read_text())
    nnpdf40_runcard = safe_load(Path("/mount/storage/Academic_Workspace/NNPDF/src/nnpdf/n3fit/NNPDF40_with_pineappl.yml").read_text())
    for d in nnpdf40_runcard["dataset_inputs"]:
        target_ds = d["dataset"]
        #if any(skipthis in target_ds for skipthis in ["HERA", "NMC", "NTV", "CHORUS", "SLAC", "BCD"]):
        #    continue
        print(target_ds)
        cfac = d.get("cfac", [])
        old_ds = API.dataset(dataset_input={"dataset": target_ds, "cfac": cfac}, theoryid=200, use_cuts="internal")
        ds = API.dataset(dataset_input={"dataset": target_ds, "cfac": cfac}, theoryid=400, use_cuts="internal")
        new_cp = central_predictions(ds, pdf)
        cp = central_predictions(old_ds, pdf)
        all_res.append(pd.concat([new_cp, cp, new_cp/cp], axis=1, keys=["vp", "pine", f"ratio {target_ds}, {cfac}"]))
        
    for i in all_res:
        mean_ratio = i[i.columns[2]].mean()
        if not (0.95 < mean_ratio < 1.05):
            print(i)

NMCPD_dw_ite

-- Reading COMMONDATA for Dataset: NMCPD_dw_ite
nData: 260 nSys: 105
-- COMMONDATA Files for NMCPD_dw_ite successfully read.

NMC

-- Reading COMMONDATA for Dataset: NMC
nData: 292 nSys: 16
-- COMMONDATA Files for NMC successfully read.

SLACP_dwsh

-- Reading COMMONDATA for Dataset: SLACP_dwsh
nData: 211 nSys: 3
-- COMMONDATA Files for SLACP_dwsh successfully read.

SLACD_dw_ite

-- Reading COMMONDATA for Dataset: SLACD_dw_ite
nData: 211 nSys: 103
-- COMMONDATA Files for SLACD_dw_ite successfully read.

BCDMSP_dwsh

-- Reading COMMONDATA for Dataset: BCDMSP_dwsh
nData: 351 nSys: 11
-- COMMONDATA Files for BCDMSP_dwsh successfully read.

BCDMSD_dw_ite

-- Reading COMMONDATA for Dataset: BCDMSD_dw_ite
nData: 254 nSys: 108
-- COMMONDATA Files for BCDMSD_dw_ite successfully read.

CHORUSNUPb_dw_ite

-- Reading COMMONDATA for Dataset: CHORUSNUPb_dw_ite
nData: 607 nSys: 1014
-- COMMONDATA Files for CHORUSNUPb_dw_ite successfully read.

CHORUSNBPb_dw_ite

-- Reading COMMONDATA 

CMS_2JET_7TEV

-- Reading COMMONDATA for Dataset: CMS_2JET_7TEV
nData: 54 nSys: 88
-- COMMONDATA Files for CMS_2JET_7TEV successfully read.

CMS_1JET_8TEV

-- Reading COMMONDATA for Dataset: CMS_1JET_8TEV
nData: 239 nSys: 293
-- COMMONDATA Files for CMS_1JET_8TEV successfully read.

CMSTTBARTOT7TEV

-- Reading COMMONDATA for Dataset: CMSTTBARTOT7TEV
nData: 1 nSys: 2
-- COMMONDATA Files for CMSTTBARTOT7TEV successfully read.

CMSTTBARTOT8TEV

-- Reading COMMONDATA for Dataset: CMSTTBARTOT8TEV
nData: 1 nSys: 2
-- COMMONDATA Files for CMSTTBARTOT8TEV successfully read.

CMSTTBARTOT13TEV

-- Reading COMMONDATA for Dataset: CMSTTBARTOT13TEV
nData: 1 nSys: 2
-- COMMONDATA Files for CMSTTBARTOT13TEV successfully read.

CMSTOPDIFF8TEVTTRAPNORM

-- Reading COMMONDATA for Dataset: CMSTOPDIFF8TEVTTRAPNORM
nData: 10 nSys: 21
-- COMMONDATA Files for CMSTOPDIFF8TEVTTRAPNORM successfully read.

CMSTTBARTOT5TEV

-- Reading COMMONDATA for Dataset: CMSTTBARTOT5TEV
nData: 1 nSys: 2
-- COMMONDATA Files fo

In [8]:
target_ds = "ATLAS_WP_JET_8TEV_PT"
cfac = ["QCD"] # ["NRM"]
old_ds = API.dataset(dataset_input={"dataset": target_ds, "cfac": cfac}, theoryid=200, use_cuts="internal")
ds = API.dataset(dataset_input={"dataset": target_ds, "cfac": cfac}, theoryid=400, use_cuts="internal")

In [4]:
# Let's try to get a prediction out of it
pdf = API.pdf(pdf="NNPDF40_nnlo_as_01180")
new_cp = central_predictions(ds, pdf)
cp = central_predictions(old_ds, pdf)
pd.concat([new_cp, cp, cp/new_cp, new_cp/cp], axis=1, keys=["pine", "vp", "ratio vp/ratio", "ratio pine/vp"])


-- Reading COMMONDATA for Dataset: ATLAS_WP_JET_8TEV_PT
nData: 16 nSys: 124
-- COMMONDATA Files for ATLAS_WP_JET_8TEV_PT successfully read.

LHAPDF 6.4.0 loading /usr/share/lhapdf/LHAPDF/NNPDF40_nnlo_as_01180/NNPDF40_nnlo_as_01180_0000.dat
NNPDF40_nnlo_as_01180 PDF set, member #0, version 1; LHAPDF ID = 331100


,pine,vp,ratio vp/ratio,ratio pine/vp
,0,0,0,0
data,,,,
1,5611.439817,5616.248344,1.000857,0.999144
2,3154.767573,3157.559479,1.000885,0.999116
3,1194.467638,1195.354244,1.000742,0.999258
4,515.187956,515.574565,1.000750,0.999250
5,246.767542,246.941654,1.000706,0.999295
6,126.644243,126.731587,1.000690,0.999311
7,70.244577,70.295367,1.000723,0.999277
8,31.544483,31.566211,1.000689,0.999312


In [5]:
pine_fkspec = ds.fkspecs[0]
old_fkspec = old_ds.fkspecs[0]

In [6]:
import pineappl
pines = [pineappl.fk_table.FkTable.read(i.as_posix()) for i in pine_fkspec.fkpath]
# Inspect the pineappl prediction
res_pine = []
pp = pines[0]
lpdf = pdf.load()

for p in pines:
    res_pine.append(p.convolute_with_one(2212, lpdf.central_member.xfxQ2))
total_pine = np.concatenate(res_pine)
print(total_pine)

LHAPDF 6.4.0 loading all 101 PDFs in set NNPDF40_nnlo_as_01180
NNPDF40_nnlo_as_01180, version 1; 101 PDF members
[0.5982222  0.53206434 0.37638204 0.13101867 0.59686953 0.51087613
 0.35425162 0.12440635 0.23262168 0.1988827  0.14422644 0.05755101
 0.01654738 0.01469668 0.01335605 0.00692297]


In [7]:
# Let's inspect the content of the old fktables, remove the cfactor for now
from validphys.fkparser import load_fktable
old_fkspec.cfactors = False
old_fktabledata = load_fktable(old_fkspec)

In [8]:
print(f"hadronic?: {old_fktabledata.hadronic}")
print(f"Q: {old_fktabledata.Q0}")
print(f"n: {old_fktabledata.ndata}")
print(f"xgrid shape: {old_fktabledata.xgrid.shape}")
#old_fktabledata.sigma

hadronic?: True
Q: 1.65
n: 16
xgrid shape: (40,)


In [9]:
# First read the metadata that vp `FKTableData` needs and that all subgrids share
Q0 = np.sqrt(pp.muf2())
xgrid = pp.x_grid()
# Hadronic means in practice that not all luminosity combinations are just electron X proton
hadronic = not all(-11 in i for i in pp.lumi())
# Now prepare the concatenation of grids
fktables = []
for p in pines:
    tmp = p.table().T/p.bin_normalizations()
    fktables.append(tmp.T)
fktable = np.concatenate(fktables, axis=0)
ndata = fktable.shape[0]

In [10]:
pp.lumi()

[(100, 21),
 (100, 203),
 (100, 208),
 (100, 200),
 (100, 103),
 (100, 108),
 (100, 115),
 (21, 21),
 (21, 203),
 (21, 208),
 (21, 200),
 (21, 103),
 (21, 108),
 (21, 115),
 (21, 100),
 (200, 203),
 (200, 208),
 (203, 203),
 (203, 208),
 (203, 200),
 (203, 103),
 (203, 108),
 (203, 115),
 (203, 100),
 (208, 208),
 (208, 200),
 (208, 103),
 (208, 108),
 (208, 115),
 (208, 100),
 (200, 200),
 (200, 103),
 (200, 108),
 (200, 115),
 (200, 100),
 (103, 103),
 (103, 108),
 (103, 115),
 (103, 100),
 (108, 108),
 (108, 115),
 (108, 100),
 (115, 115),
 (115, 100),
 (100, 100)]

In [11]:
df.columns

NameError: name 'df' is not defined

In [ ]:
# Now let's try to join the fktable, luminosity and xgrid into a pandas dataframe
# keeping compatibility with validphys and, hopefully, 50% of my own sanity

# Step 1), make the luminosity into a 14x14 mask for the evolution basis
eko_numbering_scheme = (22, 100, 21, 200, 203, 208, 215, 224, 235, 103, 108, 115, 124, 135)
# note that this is the same ordering that was used in fktables
co = []
for i, j in pp.lumi():
    # Ask where this would fall in a 14x14 matrix
    idx = eko_numbering_scheme.index(i)
    jdx = eko_numbering_scheme.index(j)
    co.append(idx*14 + jdx)
    
# Step 2) prepare the indices for the dataframe
xi = np.arange(len(xgrid))
ni = np.arange(ndata)
mi = pd.MultiIndex.from_product([ni, xi, xi], names=["data", "x1", "x2"])

# Step 3) Now play with the array until we flatten it in the right way?
# The fktables for pineappl have this extra factor of x...
# The output of pineappl is (ndata, flavours, x, x)
lf = len(co)
xfktable = fktable.reshape(ndata, lf, -1)/(xgrid[:,None]*xgrid[None,:]).flatten()
fkmod = np.moveaxis(xfktable, 1, -1)
fkframe = fkmod.reshape(-1, lf)

# Uff, big
df = pd.DataFrame(fkframe, index=mi, columns=co)

from validphys.convolution import central_hadron_predictions
from validphys.coredata import FKTableData
fk = FKTableData(sigma=df, ndata=ndata,  Q0=Q0, metadata=None, hadronic=True, xgrid=xgrid)
central_hadron_predictions(fk, pdf)

In [ ]:
# Create a luminosity tensor and check that the results are correct
from validphys.pdfbases import evolution

evol_basis = (
    "photon",
    "singlet",
    "g",
    "V",
    "V3",
    "V8",
    "V15",
    "V24",
    "V35",
    "T3",
    "T8",
    "T15",
    "T24",
    "T35",
)
total_pdf = evolution.grid_values(pdf, evol_basis, xgrid, [Q0]).squeeze()[0]/xgrid
print(total_pdf.shape)
lumi = np.einsum('ij,kl->ikjl', total_pdf, total_pdf)
lumi_masked = lumi[flavour_map]
print(fktable.shape)
print(lumi_masked.shape)
res = np.einsum('ijkl,jkl->i', fktable, lumi_masked)
#pd.concat([pd.DataFrame(res), cp, pd.DataFrame(res)/cp,  ], axis=1)

In [ ]:
xfktable.reshape(48,91,-1).shape

In [ ]:
from validphys.fkparser import open_fkpath, _parse_string, _parse_header, _build_sigma
from validphys.fkparser import _parse_flavour_map, _parse_hadronic_fast_kernel
try:
    f.close()
except:
    pass
f = open_fkpath(old_fkspec.fkpath)
line_and_stream = enumerate(f, start=1)
lineno, header = next(line_and_stream)
res = {}
while True:
    marker, header_name = _parse_header(lineno, header)
    if header_name == "FastKernel":
        break
    if header_name == "FlavourMap":
        out, lineno, header = _parse_flavour_map(line_and_stream)
    else:
        out, lineno, header = _parse_string(line_and_stream)
    res[header_name] = out   

In [ ]:
res["FlavourMap"].shape

In [ ]:
i_hate_pandas = _parse_hadronic_fast_kernel(f)

In [ ]:
i_hate_pandas

In [ ]:
old_fktabledata.sigma